# NLP Intent Recognition

Hallo und herzlich willkommen zum codecentric.AI bootcamp!

Heute wollen wir uns mit einem fortgeschrittenen Thema aus dem Bereich _natural language processing_, kurz _NLP_, genannt, beschäftigen:

> Wie bringt man Sprachassistenten, Chatbots und ähnlichen Systemen bei, die Absicht eines Nutzers aus seinen Äußerungen zu erkennen?

Dieses Problem wird im Englischen allgemein als _intent recognition_ bezeichnet und gehört zu dem ambitionierten Gebiet des _natural language understanding_, kurz _NLU_ genannt. Einen Einstieg in dieses Thema bietet das folgende [Youtube-Video](https://www.youtube.com/watch?v=H_3R8inCOvM):

In [1]:
# lade Video
from IPython.display import IFrame
IFrame('https://www.youtube.com/embed/H_3R8inCOvM', width=850, height=650)

Zusammen werden wir in diesem Tutorial mit Hilfe der NLU-Bibliothek [Rasa-NLU](https://rasa.com/docs/nlu/) einem WetterBot beibringen, einfache Fragemuster zum Wetter zu verstehen und zu beantworten. Zum Beispiel wird er auf die Fragen

> `"Wie warm war es 1989?"`

mit

<img src="img/answer-1.svg" width="85%" align="middle">

und auf

> `"Welche Temperatur hatten wir in Schleswig-Holstein und in Baden-Württemberg?"`

mit

<img src="img/answer-2.svg" width="85%" align="middle">

antworten. Der folgende Screencast gibt einen Überblick über das Notebook:

In [2]:
# lade Video
from IPython.display import IFrame
IFrame('https://www.youtube.com/embed/pVwO4Brs4kY', width=850, height=650)

Damit es gleich richtig losgehen kann, importieren wir noch zwei Standardbibliotheken und vereinbaren das Datenverzeichnis:

In [3]:
import os
import numpy as np


DATA_DIR = 'data'

## Unser Ausgangspunkt

Allgemein ist die Aufgabe, aus einer Sprachäußerung die zugrunde liegende Absicht zu erkennen, selbst für Menschen manchmal nicht einfach. Soll ein Computer diese schwierige Aufgabe lösen, so muss man sich überlegen, was man zu einem gegebenen Input &mdash; also einer (unstrukturierten) Sprachäußerung &mdash; für einen Output erwarten, wie man also Absichten modelliert und strukturiert.

Weit verbreitet ist folgender Ansatz für Intent Recognition:

- jede Äußerung wird einer _Domain_, also einem Gebiet, zugeordnet,
- für jede _Domain_ gibt es einen festen Satz von _Intents_, also eine Reihe von Absichten,
- jede Absicht kann durch _Parameter_ konkretisiert werden und hat dafür eine Reihe von _Slots_, die wie Parameter einer Funktion oder Felder eines Formulares mit gewissen Werten gefüllt werden können.

Für die Äußerungen

>  - `"Wie warm war es 1990 in Berlin?"`
>  - `"Welche Temperatur hatten wir in Hessen im Jahr 2018?"`
>  - `"Wie komme ich zum Hauptbahnhof?"`

könnte _Intent Recognition_ also zum Beispiel jeweils folgende Ergebnisse liefern:

> - `{'intent': 'Frag_Temperatur', 'slots': {'Ort': 'Berlin', 'Jahr': '1990'}}`
> - `{'intent': 'Frag_Temperatur', 'slots': {'Ort': 'Hessen', 'Jahr': '2018'}}`
> - `{'intent': 'Frag_Weg', 'slots': {'Start': None, 'Ziel': 'Hauptbahnhof'}}`

Für Python steht eine ganze von NLP-Bibliotheken zur Verfügung, die Intent Recognition in der einen oder anderen Form ermöglichen, zum Beispiel

- [Rasa NLU](https://rasa.com/docs/nlu/) (&bdquo;Language Understanding for chatbots and AI assistants&ldquo;),
- [snips](https://snips-nlu.readthedocs.io/en/latest/) (&bdquo;Using Voice to Make Technology Disappear&ldquo;),
- [DeepPavlov](http://deeppavlov.ai) (&bdquo;an open-source conversational AI library&ldquo;),
- [NLP Architect](http://nlp_architect.nervanasys.com/index.html) von Intel (&bdquo;for exploring state-of-the-art deep learning topologies and techniques for natural language processing and natural language unterstanding&ldquo;),
- [pytext](https://pytext-pytext.readthedocs-hosted.com/en/latest/index.html) von Facebook (&bdquo;a deep-learning based NLP modeling framework built on PyTorch&ldquo;).

Wir entscheiden uns im Folgenden für die Bibliothek Rasa NLU, weil wir dafür bequem mit einem Open-Source-Tool (chatette) umfangreiche Trainingsdaten generieren können. Rasa NLU wiederum benutzt  die NLP-Bibliothek [spaCy](https://spacy.io), die Machine-Learning-Bibliothek [scikit-learn](https://scikit-learn.org/stable/) und die Deep-Learning-Bibliothek [TensorFlow](https://www.tensorflow.org/).


## Intent Recognition von Anfang bis Ende mit Rasa NLU

Schauen wir uns an, wie man eine Sprach-Engine für Intent Recognition trainieren kann! Dafür beschränken wir uns zunächst auf wenige Intents und Trainingsdaten und gehen die benötigten Schritte von Anfang bis Ende durch.

### Schritt 1: Intents durch Trainingsdaten beschreiben

Als Erstes müssen wir die Intents mit Hilfe von Trainingsdaten beschreiben. _Rasa NLU_ erwartet beides zusammen in einer Datei im menschenfreundlichen [Markdown-Format](http://markdown.de/) oder im computerfreundlichen [JSON-Format](https://de.wikipedia.org/wiki/JavaScript_Object_Notation). Ein Beispiel für solche Trainingsdaten im Markdown-Format ist der folgende Python-String, den wir in die Datei `intents.md` speichern: 

In [4]:
TRAIN_INTENTS = """
## intent: Frag_Temperatur
- Wie [warm](Eigenschaft) war es [1900](Zeit) in [Brandenburg](Ort)
- Wie [kalt](Eigenschaft) war es in [Hessen](Ort) [1900](Zeit)
- Was war die Temperatur [1977](Zeit) in [Sachsen](Ort)

## intent: Frag_Ort
- Wo war es [1998](Zeit) am [kältesten](Superlativ:kalt)
- Finde das [kältesten](Superlativ:kalt) Bundesland im Jahr [2004](Zeit)
- Wo war es [2010](Zeit) [kälter](Komparativ:kalt) als [1994](Zeit) in [Rheinland-Pfalz](Ort)

## intent: Frag_Zeit
- Wann war es in [Bayern](Ort) am [kühlsten](Superlativ:kalt)
- Finde das [kälteste](Superlativ:kalt) Jahr im [Saarland](Ort)
- Wann war es in [Schleswig-Holstein](Ort) [wärmer](Komparativ:warm) als in [Baden-Württemberg](Ort)

## intent: Ende
- Ende
- Auf Wiedersehen
- Tschuess
"""


INTENTS_PATH = os.path.join(DATA_DIR, 'intents.md')


def write_file(filename, text):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

write_file(INTENTS_PATH, TRAIN_INTENTS)

Hier wird jeder Intent erst in der Form

> `## intent: NAME`

deklariert, wobei `NAME` durch die Bezeichnung des Intents zu ersetzen ist. Anschließend wird der Intent durch eine Liste von
Beispiel-Äußerungen beschrieben. Die Parameter beziehungsweise Slots werden in den Beispieläußerungen in der Form

> `[WERT](SLOT)`

markiert, wobei `SLOT` die Bezeichnung des Slots und `Wert` der entsprechende Teil der Äußerung ist.


### Schritt 2: Sprach-Engine konfigurieren...

Die Sprach-Engine von _Rasa NLU_ ist als Pipeline gestaltet und [sehr flexibel konfigurierbar](https://rasa.com/docs/nlu/components/#section-pipeline). Zwei [Beispiel-Konfigurationen](https://rasa.com/docs/nlu/choosing_pipeline/) sind in Rasa bereits enthalten:

- `spacy_sklearn` verwendet vortrainierte Wortvektoren, eine [scikit-learn-Implementierung](https://scikit-learn.org/stable/modules/svm.html) einer linearen [Support-vector Machine]( https://en.wikipedia.org/wiki/Support-vector_machine) für die Klassifikation und wird für kleine Trainingsmengen (<1000) empfohlen. Da diese Pipeline vortrainierte Wortvektoren und spaCy benötigt, kann sie nur für [die meisten westeuropäische Sprachen](https://rasa.com/docs/nlu/languages/#section-languages) verwendet werden. Allerdings sind die Version 0.20.1 von scikit-learn und 0.13.8 von Rasa-NLU nicht kompatibel

- `tensorflow_embedding` trainiert für die Klassifikation Einbettungen von Äußerungen und von Intents in denselben Vektorraum  und wird für größere Trainingsmengen (>1000) empfohlen. Die zu Grunde liegende Idee stammt aus dem Artikel [StarSpace: Embed All The Things!](https://arxiv.org/abs/1709.03856). Sie ist sehr vielseitig anwendbar und beispielsweise auch für  [Question Answering](https://en.wikipedia.org/wiki/Question_answering) geeignet. Diese Pipeline benötigt kein Vorwissen über die verwendete Sprache, ist also universell einsetzbar, und kann auch auf das Erkennen mehrerer Intents in einer Äußerung trainiert werden.

Zum Füllen der Slots verwenden beide Pipelines eine [Python-Implementierung](http://www.chokkan.org/software/crfsuite/) von [Conditional Random Fields](https://en.wikipedia.org/wiki/Conditional_random_field).

Die Konfiguration der Pipeline wird durch eine YAML-Datei beschrieben. Der folgende Python-String entspricht der Konfiguration `tensorflow_embedding`:

In [5]:
CONFIG_TF = """
pipeline:
- name: "tokenizer_whitespace"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"
"""

### Schritt 3: ...trainieren...

Sind die Trainingsdaten und die Konfiguration der Pipeline beisammen, so kann die Sprach-Engine trainiert werden. In der Regel erfolgt dies bei Rasa mit Hilfe eines Kommandozeilen-Interface oder direkt [in Python](https://rasa.com/docs/nlu/python/). Die folgende Funktion `train` erwartet die Konfiguration als Python-String und den Namen der Datei mit den Trainingsdaten und gibt die trainierte Sprach-Engine als Instanz einer `Interpreter`-Klasse zurück:

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import rasa_nlu.training_data
import rasa_nlu.config
from rasa_nlu.model import Trainer, Interpreter

MODEL_DIR = 'models'

def train(config=CONFIG_TF, intents_path=INTENTS_PATH):
    config_path = os.path.join(DATA_DIR, 'rasa_config.yml')
    write_file(config_path, config)
    trainer = Trainer(rasa_nlu.config.load(config_path))
    trainer.train(rasa_nlu.training_data.load_data(intents_path))
    return Interpreter.load(trainer.persist(MODEL_DIR))

interpreter = train()

/usr/local/lib/python3.6/site-packages/rasa_nlu/utils/__init__.py:239: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(read_file(filename, "utf-8"))
Epochs: 100%|██████████| 300/300 [00:02<00:00, 146.06it/s, loss=0.097, acc=1.000]


INFO:tensorflow:Restoring parameters from /notebooks/nlp_intent/models/default/model_20191006-125739/intent_classifier_tensorflow_embedding.ckpt


### Schritt 4: ...und testen!

Wir testen nun, ob die Sprach-Engine `interpreter` folgende Test-Äußerungen richtig versteht:

In [7]:
TEST_UTTERANCES = [
    'Was war die durchschnittliche Temperatur 2004 in Mecklenburg-Vorpommern',
    'Nenn mir das wärmste Bundesland 2018',
    'In welchem Jahr war es in Nordrhein-Westfalen heißer als 1990',
    'Wo war es 2000 am kältesten',
    'Bis bald',
]

Die Methode `parse` von `interpreter` erwartet eine Äußerung als Python-String, wendet Intent Recognition an und liefert eine sehr detaillierte Rückgabe:

In [8]:
interpreter.parse(TEST_UTTERANCES[0])

{'intent': {'name': 'Frag_Temperatur', 'confidence': 0.9676849842071533},
 'entities': [{'start': 41,
   'end': 45,
   'value': '2004',
   'entity': 'Zeit',
   'confidence': 0.8843208566489816,
   'extractor': 'ner_crf'},
  {'start': 49,
   'end': 71,
   'value': 'Mecklenburg-Vorpommern',
   'entity': 'Ort',
   'confidence': 0.9251510709571028,
   'extractor': 'ner_crf'}],
 'intent_ranking': [{'name': 'Frag_Temperatur',
   'confidence': 0.9676849842071533},
  {'name': 'Ende', 'confidence': 0.0},
  {'name': 'Frag_Zeit', 'confidence': 0.0},
  {'name': 'Frag_Ort', 'confidence': 0.0}],
 'text': 'Was war die durchschnittliche Temperatur 2004 in Mecklenburg-Vorpommern'}

Die Rückgabe umfasst im Wesentlichen

- den Namen des ermittelten Intent sowie eine Sicherheit beziehungsweise Konfidenz zwischen 0 und 1,
- für jeden ermittelten Parameter die Start- und Endposition in der Äußerung, den Wert und wieder eine Konfidenz,
- ein Ranking der möglichen Intents nach der Sicherheit/Konfidenz, mit der sie in dieser Äußerung vermutet wurden.

Für eine übersichtlichere Darstellung und leichte Weiterverarbeitung bereiten wir die Rückgabe mit Hilfe der Funktionen `extract_intent` und `extract_confidences` ein wenig auf. Anschließend gehen wir unsere Test-Äußerungen durch:

In [9]:
def extract_intent(intent):
    return (intent['intent']['name'] if intent['intent'] else None,
            [(ent['entity'], ent['value']) for ent in intent['entities']])


def extract_confidences(intent):
    return (intent['intent']['confidence'] if intent['intent'] else None,
           [ent['confidence'] for ent in intent['entities']])


def test(interpreter, utterances=TEST_UTTERANCES):
    for utterance in utterances:
        intent = interpreter.parse(utterance)
        print('<', utterance)
        print('>', extract_intent(intent))
        print(' ', extract_confidences(intent))
        print()

test(interpreter)

< Was war die durchschnittliche Temperatur 2004 in Mecklenburg-Vorpommern
> ('Frag_Temperatur', [('Zeit', '2004'), ('Ort', 'Mecklenburg-Vorpommern')])
  (0.9676849842071533, [0.8843208566489816, 0.9251510709571028])

< Nenn mir das wärmste Bundesland 2018
> ('Frag_Ort', [('Zeit', '2018')])
  (0.9345992803573608, [0.8188949907951559])

< In welchem Jahr war es in Nordrhein-Westfalen heißer als 1990
> ('Frag_Zeit', [('Ort', 'Nordrhein-Westfalen'), ('Komparativ', 'heißer'), ('Zeit', '1990')])
  (0.5457459688186646, [0.8231770104558299, 0.5422260768239187, 0.8920504023927753])

< Wo war es 2000 am kältesten
> ('Frag_Ort', [('Zeit', '2000'), ('Superlativ', 'kalt')])
  (0.9631474018096924, [0.8707385652923301, 0.9376147531759642])

< Bis bald
> (None, [])
  (0.0, [])



Das Ergebnis ist noch nicht ganz überzeugend &mdash;  wir haben aber auch nur ganz wenig Trainingsdaten vorgegeben!

##  Trainingsdaten generieren mit Chatette

Für ein erfolgreiches Training brauchen wir also viel mehr Trainingsdaten. Doch fängt man an, weitere Beispiele aufzuschreiben, so fallen einem schnell viele kleine Variationsmöglichkeiten ein, die sich recht frei kombinieren lassen. Zum Beispiel können wir für eine Frage nach der Temperatur in Berlin im Jahr 1990 mit jeder der Phrasen
> - "Wie warm war es..."
> - "Wie kalt war es..."
> - "Welche Temperatur hatten wir..."

beginnen und dann mit

> - "...in Berlin 1990"
> - "...1990 in Berlin"

abschließen, vor "1990" noch "im Jahr" einfügen und so weiter. Statt alle denkbaren Kombinationen aufzuschreiben, ist es sinnvoller, die Möglichkeiten mit Hilfe von Regeln zu beschreiben und daraus Trainingsdaten generieren zu lassen. Genau das ermöglicht das Python-Tool [chatette](https://github.com/SimGus/Chatette), das wir im Folgenden verwenden. Dieses Tool liest Regeln, die einer speziellen Syntax folgen müssen, aus einer Datei aus und erzeugt dann daraus Trainingsdaten für Rasa NLU im JSON-Format.


### Regeln zur Erzeugung von Trainingsdaten

Wir legen im Folgenden erst einen Grundvorrat an Regeln für die Intents `Frag_Temperatur`, `Frag_Ort`, `Frag_Zeit` und `Ende` in einem Python-Dictionary an und erläutern danach genauer, wie die Regeln aufgebaut sind:

In [10]:
RULES = {
    '@[Ort]': (
        'Brandenburg', 'Baden-Wuerttemberg', 'Bayern', 'Hessen',
        'Rheinland-Pfalz', 'Schleswig-Holstein', 'Saarland', 'Sachsen',
    ),
    '@[Zeit]': set(map(str, np.random.randint(1891, 2018, size=5))),
    '@[Komparativ]': ('wärmer', 'kälter',),
    '@[Superlativ]': ('wärmsten', 'kältesten',),
    '%[Frag_Temperatur]': ('Wie {warm/kalt} war es ~[zeit_ort]',
                  'Welche Temperatur hatten wir ~[zeit_ort]',
                  'Wie war die Temperatur ~[zeit_ort]',
    ),
    '%[Frag_Ort]': (
        '~[wo_war] es @[Zeit] @[Komparativ] als {@[Zeit]/in @[Ort]}',
        '~[wo_war] es @[Zeit] am @[Superlativ]',
    ),
    '%[Frag_Jahr]': (
        '~[wann_war] es in @[Ort] @[Komparativ] als {@[Zeit]/in @[Ort]}',
        '~[wann_war] es in @[Ort] am @[Superlativ]',
    ),
    '%[Ende]': ('Ende', 'Auf Wiedersehen', 'Tschuess',),
    '~[finde]': ('Sag mir', 'Finde'),
    '~[wie_war]': ('Wie war', '~[finde]',),
    '~[was_war]': ('Was war', '~[finde]',),
    '~[wo_war]': ('Wo war', 'In welchem {Bundesland|Land} war',),
    '~[wann_war]': ('Wann war', 'In welchem Jahr war',),
    '~[zeit_ort]': ('@[Zeit] in @[Ort]', '@[Ort] in @[Zeit]',),
    '~[Bundesland]': ('Land', 'Bundesland',),
}

Jede Regel besteht aus einem Namen beziehungsweise Platzhalter und einer Menge von Phrasen. Je nachdem, ob der Name die Form
> `%[NAME]`, `@[NAME]` oder `~[NAME]`

hat, beschreibt die Regel einen

> _Intent_, _Slot_ oder eine _Alternative_

mit der Bezeichnung `NAME`.  Jede Phrase kann ihrerseits Platzhalter für Slots und Alternativen erhalten. Diese Platzhalter werden bei der Erzeugung von Trainingsdaten von chatette jeweils durch eine der Phrasen ersetzt, die in der Regel für den jeweiligen Slot beziehungsweise die Alternativen aufgelistet sind. Außerdem können Phrasen

- Alternativen der Form `{_|_|_}`,
- optionale Teile in der Form `[_?]`

und einige weitere spezielle Konstrukte enthalten. Mehr Details finden sich in der [Syntax-Beschreibung](https://github.com/SimGus/Chatette/wiki/Syntax-specifications) von chatette.



### Erzeugung der Trainingsdaten

Die in dem Python-Dictionary kompakt abgelegten Regeln müssen nun für chatette so formatiert werden, dass bei jeder Regel der Name einen neuen Absatz einleitet und anschließend die möglichen Phrasen schön eingerückt Zeile für Zeile aufgelistet werden. Dies leistet die folgende Funktion `format_rules`.  Zusätzlich fügt sie eine Vorgabe ein, wieviel Trainingsbeispiele pro Intent erzeugt werden sollen:

In [11]:
def format_rules(rules, train_samples):
    train_str =  "('training':'{}')".format(train_samples)
    llines = [[name if (name[0] != '%') else name + train_str]
              + ['    ' + val for val in rules[name]] + [''] for name in rules]
    return '\n'.join((l for lines in llines for l in lines))


Nun wenden wir chatette an, um die Trainingsdaten zu generieren. Dafür bietet chatette ein bequemes [Kommandozeilen-Interface](https://github.com/SimGus/Chatette/wiki/Command-line-interface), aber wir verwenden direkt die zu Grunde liegenden Python-Module.

Die folgende Funktion `chatette` erwartet wie `format_rules` ein Python-Dictionary mit Regeln, schreibt diese passend formatiert in eine Datei, löscht etwaige zuvor generierte Trainingsdateien und erzeugt dann den Regeln entsprechend neue Trainingsdaten.

In [12]:
from chatette.adapters import RasaAdapter
from chatette.parsing import Parser
from chatette.generator import Generator
import glob

TRAIN_SAMPLES = 400
CHATETTE_DIR = os.path.join(DATA_DIR, 'chatette')


def chatette(rules=RULES, train_samples=TRAIN_SAMPLES):
    rules_path = os.path.join(DATA_DIR, 'intents.chatette')
    write_file(rules_path, format_rules(rules, train_samples))
    with open(rules_path, 'r') as rule_file:
        parser = Parser(rule_file)
        parser.parse()
    generator = Generator(parser)
    for f in glob.glob(os.path.join(CHATETTE_DIR, '*')):
        os.remove(f)
    RasaAdapter().write(CHATETTE_DIR, list(generator.generate_train()),
                        generator.get_entities_synonyms())
    
chatette(train_samples=400)

[DBG] Parsing file: data/intents.chatette
[DBG] Parsing of file: data/intents.chatette finished
[DBG] Generating training examples...


### Und nun: neuer Test!

Bringen die umfangreicheren Trainingsdaten wirklich eine Verbesserung? Schauen wir's uns an! Um verschiedene Sprach-Engines zu vergleichen, nutzen wir die folgende Funktion:

In [13]:
def train_and_test(config=CONFIG_TF, utterances=TEST_UTTERANCES):
    interpreter = train(config, CHATETTE_DIR)
    test(interpreter, utterances)
    return interpreter

interpreter = train_and_test()

/usr/local/lib/python3.6/site-packages/rasa_nlu/utils/__init__.py:239: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(read_file(filename, "utf-8"))
Epochs: 100%|██████████| 300/300 [00:21<00:00, 13.93it/s, loss=0.003, acc=1.000]
/usr/local/lib/python3.6/site-packages/rasa_nlu/extractors/entity_synonyms.py:85: UserWarning: Failed to load synonyms file from '/notebooks/nlp_intent/models/default/model_20191006-125804/entity_synonyms.json'
  "".format(entity_synonyms_file))


INFO:tensorflow:Restoring parameters from /notebooks/nlp_intent/models/default/model_20191006-125804/intent_classifier_tensorflow_embedding.ckpt
< Was war die durchschnittliche Temperatur 2004 in Mecklenburg-Vorpommern
> ('Frag_Temperatur', [('Zeit', '2004'), ('Ort', 'Mecklenburg-Vorpommern')])
  (0.9136318564414978, [0.8541794001234732, 0.9875637470644905])

< Nenn mir das wärmste Bundesland 2018
> ('Frag_Ort', [('Zeit', '2018')])
  (0.6670867204666138, [0.9205167054194056])

< In welchem Jahr war es in Nordrhein-Westfalen heißer als 1990
> ('Frag_Jahr', [('Ort', 'Nordrhein-Westfalen'), ('Komparativ', 'heißer'), ('Zeit', '1990')])
  (0.9601661562919617, [0.968290681457331, 0.9009353187976876, 0.9976314157284462])

< Wo war es 2000 am kältesten
> ('Frag_Ort', [('Zeit', '2000'), ('Superlativ', 'kältesten')])
  (0.9656134247779846, [0.889369756926878, 0.9902646526673686])

< Bis bald
> (None, [])
  (0.0, [])



Hier wurde nur die letzte Äußerung nicht verstanden, aber das ist auch nicht weiter verwunderlich.

##  Unser kleiner WetterBot

Experimentieren macht mehr Spaß, wenn es auch mal zischt und knallt. Oder zumindest irgendeine andere Reaktion erfolgt. Und deswegen bauen wir uns einen kleinen WetterBot, der auf die erkannten Intents reagieren kann. Zuerst schreiben wir dafür eine Eingabe-Verarbeitungs-Ausgabe-Schleife. Diese erwartet als Parameter erstens die Sprach-Engine `interpreter` und zweitens ein Python-Dictionary `handlers`, welches jeder Intent-Bezeichnung einen Handler zuordnet. Der Handler wird dann mit dem erkannten Intent aufgerufen und sollte zurückgeben, ob die Schleife fortgeführt werden soll oder nicht:

In [14]:
def dialog(interpreter, handlers):
    quit = False
    while not quit:
        intent = extract_intent(interpreter.parse(input('>')))
        print('<', intent)
        intent_name = intent[0]
        if intent_name in handlers:
            quit = handlers[intent_name](intent)


Wir implementieren gleich beispielhaft einen Handler für den Intent `Frag_Temperatur`und reagieren auf alle anderen Intents mit einer Standard-Antwort:

In [15]:
def message(msg, quit=False):
    print(msg)
    return quit

HANDLERS = { 
    'Ende': lambda intent: message('=> Oh, wie schade. Bis bald!', True),
    'Frag_Zeit': lambda intent: message('=> Das ist eine gute Frage.'),
    'Frag_Ort': lambda intent: message('=> Dafür wurde ich nicht programmiert.'),
    'Frag_Temperatur': lambda intent: message('=> Das weiss ich nicht.')
}


Um die Fragen nach den Temperaturen zu beantworten, nutzen wir [Archiv-Daten](ftp://ftp-cdc.dwd.de/pub/CDC/regional_averages_DE/annual/air_temperature_mean/regional_averages_tm_year.txt) des [Deutschen Wetterdienstes](https://www.dwd.de), die wir schon etwas aufbereitet haben. Die Routine `show` gibt die nachgefragten Temperaturdaten je nach Anzahl der angegebenen Jahre und Bundesländer als Liniendiagramm, Balkendiagramm oder in Textform an. Der eigentliche Hander `frag_wert` prüft, ob die angegebenen Jahre und Orte auch zulässig sind und setzt, falls eine der beiden Angaben fehlt, einfach alle Jahre beziehungsweise Bundesländer ein:

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

sns.set()

DATA_PATH = os.path.join(DATA_DIR, 'temperaturen.txt')
temperature = pd.read_csv(DATA_PATH, index_col=0, sep=';')

def show(times, places):
    if (len(places) == 0) and (len(times) == 0):
        print('Keine zulässigen Orte oder Zeiten')
    elif (len(places) == 1) and (len(times) == 1):
        print(temperature.loc[times, places])
    else:
        if (len(places) > 1) and (len(times) == 1):
            temperature.loc[times[0], places].plot.barh()
        if (len(places) == 1) and (len(times) > 1):
            temperature.loc[times, places[0]].plot.line()
        if (len(places) > 1) and (len(times) > 1):
            temperature.loc[times, places].plot.line()
            plt.legend(bbox_to_anchor=(1.05,1), loc=2, borderaxespad=0.)
        plt.show()

def frag_temperatur(intent):
    def validate(options, ent_name, fn):
        chosen = [fn(value) for (name, value) in intent[1] if name == ent_name]
        return list(set(options) & set(chosen)) if chosen else options
    places = validate(list(temperature.columns), 'Ort', lambda x:x)
    times = validate(list(temperature.index), 'Zeit', int)
    show(times, places)
    return False

HANDLERS['Frag_Temperatur'] = frag_temperatur

Nun kann der WetterBot getestet werden! Zum Beispiel mit

>  "Wie warm war es in Baden-Württemberg und Sachsen?"

In [17]:
dialog(interpreter, HANDLERS)

> 


< (None, [])


> import os os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


< ('Ende', [])
=> Oh, wie schade. Bis bald!


## Intent Recognition selbst gemacht &mdash; ein Bi-LSTM-Netzwerk mit Keras

Im Prinzip haben wir nun gesehen, wie sich Intent Recognition mit Hilfe von Rasa NLU recht einfach anwenden lässt. Aber wie funktioniert das ganz genau? In diesem zweiten Teil des Notebooks werden wir

  - ein bidirektionales rekurrentes Netz, wie es im Video vorgestellt wurde, implementieren,
  - die mit chatette erstellten Trainingsdaten so aufbereiten, dass wir damit das Netz trainieren können,

und sehen, dass das ganz gut klappt und gar nicht so schwer ist!

### Intents einlesen und aufbereiten

Zuerst lesen wir die Trainings-Daten, die von chatette im JSON-Format ausgegeben in die Date `RASA_INTENTS` geschrieben wurden, aus, und schauen uns das Format der Einträge an:


In [18]:
import json

CHATETTE_DIR = os.path.join(DATA_DIR, 'chatette')
RASA_INTENTS = os.path.join(CHATETTE_DIR, 'output.json')


def load_intents():
    with open(RASA_INTENTS) as intents_file:
        intents = json.load(intents_file)
    return intents['rasa_nlu_data']['common_examples']


sample_intent = load_intents()[0]

Wie bereits im [Video](https://www.youtube.com/watch?v=H_3R8inCOvM) erklärt, sind für Intent Recognition zwei Aufgaben zu lösen:

  - die _Klassifikation_ des Intent anhand der gegebenen Äußerung und
  - das Füllen der Slots.

Die zweite Aufgabe kann man als _Sequence Tagging_ auffassen &mdash; für jedes Token der Äußerung ist zu bestimmen, ob es den Parameter für einen Slot darstellt oder nicht. Für den Beispiel-Intent

>   `{'entities': [{'end': 20, 'entity': 'Zeit', 'start': 16, 'value': '1993'},
>                  {'end': 35, 'entity': 'Ort', 'start': 24, 'value': 'Brandenburg'}],
>     'intent': 'Frag_Temperatur',
>     'text': 'Wie warm war es 1993 in Brandenburg'}`

wäre die Eingabe für diese beiden Aufgaben also die Token-Folge

> `['Wie', 'warm', 'war', 'es', '1993', 'in', 'Brandenburg']`

und die gewünschte Ausgabe jeweils

> `'Frag_Temperatur'`

beziehungsweise die Tag-Folge

> `['-', '-', '-', '-', 'Zeit', '-', 'Ort']`

Die folgende Funktion extrahiert aus den geladenen Beispiel-Intents die gewünschte Eingabe und die Ausgaben für diese beiden Aufgaben:

In [19]:
import spacy
from itertools import accumulate

nlp = spacy.load('de_core_news_sm')

def tokenize(text):
    return [word for word in nlp(text)]

NO_ENTITY = '-'

def intent_and_sequences(intent):
    def get_tag(offset):
        """Returns the tag (+slot name) for token starting at `offset`"""
        ents = [ent['entity'] for ent in intent['entities'] if ent['start'] == offset]
        return ents[0] if ents else NO_ENTITY
    token = tokenize(intent['text'])
    # `offsets` is the list of starting positions of the token
    offsets = list(accumulate([0,] + [len(t.text_with_ws) for t in token]))
    return (intent['intent'], token, list(map(get_tag, offsets[:-1])))

intent_and_sequences(sample_intent)


('Frag_Temperatur',
 [Wie, warm, war, es, 1977, in, Brandenburg],
 ['-', '-', '-', '-', 'Zeit', '-', 'Ort'])

### Symbolische Daten in numerische Daten umwandeln

Die aufbereiteten Intents enthalten nun jeweils

  1. die Folge der Token als "Eingabe"
  2. den Namen des Intent als Ergebnis der Klassifikation und
  3. die Folge der Slot-Namen als Ergebnis des Sequence Tagging.

Diese kategoriellen Daten müssen wir für die Weiterverarbeitung in numerische Daten umwandeln. Dafür bieten sich

  - für 1. Wortvektoren und
  - für 2. und 3. die One-hot-Kodierung an.

Außerdem müssen wir die Eingabe-Folge und Tag-Folge auf eine feste Länge bringen.

Beginnen wir mit der One-hot-Kodierung. Die folgende Funktion erzeug zu einer gegebenen Menge von Objekten ein Paar von Python-Dictionaries, welche jedem Objekt einen One-hot-Code und umgekehrt jedem Index das entsprechende Objekt zuordnet.


In [20]:
def ohe(s):
    codes = np.eye(len(s))
    numerated = list(enumerate(s))
    return ({value: codes[idx] for (idx, value) in numerated},
            {idx: value for (idx, value) in numerated})


Die nächste Hilfsfunktion erwartet eine Liste von Elementen und schneidet diese auf eine vorgegebene Länge beziehungsweise füllt sie mit einem vorgegebenen Element auf diese Länge auf.

In [21]:
def fill(items, max_len, filler):
    if len(items) < max_len:
        return items + [filler] * (max_len - len(items))
    else:
        return items[0:max_len]


Die Umwandlung der aufbereiteten Intent-Tripel in numerische Daten verpacken wir in einen [scikit-learn-Transformer](https://scikit-learn.org/stable/data_transforms.html), weil während der Umwandlung die One-Hot-Kodierung der Intent-Namen und Slot-Namen gelernt und eventuell später für neue Testdaten wieder gebraucht wird.

In [22]:
from sklearn.base import BaseEstimator, TransformerMixin

MAX_LEN = 20
VEC_DIM = len(list(nlp(' '))[0].vector)

class IntentNumerizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        intent_names = set((x[0] for x in X))
        self.intents_ohe, self.idx_intents = ohe(intent_names)
        self.nr_intents = len(intent_names)
        tag_lists = list(map(lambda x: set(x[2]), X)) + [[NO_ENTITY]]
        tag_names = frozenset().union(*tag_lists)
        # tag_names = set(())
        self.tags_ohe, self.idx_tags = ohe(tag_names)
        self.nr_tags = len(tag_names)
        return self

    def transform_utterance(self, token):
        return np.stack(fill([tok.vector for tok in token], MAX_LEN,
                             np.zeros((VEC_DIM))))

    def transform_tags(self, tags):
        return np.stack([self.tags_ohe[t] for t in fill(tags, MAX_LEN, NO_ENTITY)]) 

    def transform(self, X):
        return (np.stack([self.transform_utterance(x[1]) for x in X]),
                np.stack([self.intents_ohe[x[0]] for x in X]),
                np.stack([self.transform_tags(x[2]) for x in X]))

    def revert(self, intent_idx, tag_idxs):
        return (self.idx_intents[intent_idx],
                [self.idx_tags[t] for t in tag_idxs])


### Keras-Implementierung eines Bi-LSTM-Netzes für Intent Recognition

Wir implementieren nun mit Keras eine Netz-Architektur, die in [diesem Artikel]() vorgeschlagen wurde und schematisch in folgendem Diagramm dargestellt ist:

<img src="img/birnn.svg"  style="background:white" width="80%" align="middle">

Hierbei wird

1. die Eingabe, wie bereits erklärt, als Folge von Wortvektoren dargestellt,
2. diese Eingabe erst durch eine rekurrente Schicht forwärts abgearbeitet,
3. der Endzustand dieser Schicht als Initialisierung einer sich anschließenden rekurrenten Schicht verwendet, welche die Eingabefolge rückwärts abarbeitet,
4. der Endzustand dieser Schicht an eine Schicht mit genau so vielen Neuronen, wie es Intent-Klassen gibt, zur Klassifikation des Intent weitergleitet,
5. die Ausgabe der beiden rekurrenten Schichten für jeden Schritt zusammengefügt und
6. die zusammengefügte Ausgabe jeweils an ein Bündel von so vielen Neuronen, wie es Slot-Arten gibt, zur Klassifikation des Tags des jeweiligen Wortes weitergeleitet.

Genau diesen Aufbau bilden wir nun mit Keras ab, wobei wir die [funktionale API]() benutzen. Als Loss-Funktion verwenden wir jeweils [kategorielle Kreuzentropie](). Für die rekurrenten Schichten verwenden wir [LSTM-Zellen](), auf die wir gleich noch eingehen.

In [23]:
from keras.models import Model
from keras.layers import Input, LSTM, Concatenate, TimeDistributed, Dense

UNITS = 256

def build_bilstm(input_dim, nr_intents, nr_tags, units=UNITS):
    inputs = Input(shape=(MAX_LEN, input_dim))
    lstm_params = {'units': units, 'return_sequences': True, 'return_state': True}
    fwd = LSTM(**lstm_params)(inputs)
    bwd = LSTM(**lstm_params)(inputs, initial_state=fwd[1:])
    merged = Concatenate()([fwd[0], bwd[0]])
    tags = TimeDistributed(Dense(nr_tags, activation='softmax'))(merged)
    intent = Dense(nr_intents, activation='softmax')(bwd[2])
    model = Model(inputs=inputs, outputs=[intent, tags])
    model.compile(optimizer='Adam' ,loss='categorical_crossentropy')
    return model


Using TensorFlow backend.


Schauen wir uns einmal genauer an, wie so eine LSTM-Zelle aufgebaut ist:

<img src="img/lstm.svg" style="background:white" width="70%" align="middle">


Die Bezeichnung 'LSTM' steht für _long short-term memory_ und rührt daher, dass solch eine Zelle neben der Eingabe des aktuellen Schrittes nicht nur die Ausgabe des vorherigen Schrittes, sondern zusätzlich auch einen Speicherwert des vorherigen Schrittes erhält. Nacheinander wird in der LSTM-Zelle dann jeweils anhand der aktuellen Eingabe und der vorherigen Ausgabe

1. in einem _forget gate_ entschieden, wieviel vom alten Speicherwert vergessen werden soll,
2. in einem _input gate_ entschieden, wieviel von der neuen Eingabe in den neuen Speicherwert aufgenommen werden soll,
3. in einem _output gate_ aus dem aktuellen Speicher die aktuelle Ausgabe gebildet.

### Training und Test des Bi-LSTM-Netzes

Schauen wir uns nun an, wie gut das funktioniert! Dazu müssen wir nun alles zusammenfügen und tun das in zwei Schritten.

Die Funktion `train_test_data` erwartet als Eingabe Regeln, wie wir sie für chatette in einem Python-Dictionary gespeichert hatten, und liefert die entsprechend erzeugten Intents in numerisch aufbereiter Form, aufgeteilt in Trainings- und Validierungsdaten, einschließlich des angepassten `IntentNumerizer`zurück.

In [24]:
TRAIN_RATIO = 0.7

def train_test_data(rules=RULES, train_ratio=TRAIN_RATIO):
    structured_intents = list(map(intent_and_sequences, load_intents()))
    intent_numerizer = IntentNumerizer()
    X, y, Y = intent_numerizer.fit_transform(structured_intents)
    nr_samples = len(y)
    shuffled_indices = np.random.permutation(nr_samples)
    split = int(nr_samples * train_ratio)
    train_indices, test_indices = (shuffled_indices[0:split], shuffled_indices[split:])
    y_train, X_train, Y_train = y[train_indices], X[train_indices], Y[train_indices]
    y_test, X_test, Y_test = y[test_indices], X[test_indices], Y[test_indices]
    return intent_numerizer, X_train, y_train, Y_train, X_test, y_test, Y_test


Mit diesen Trainings- und Testdaten trainiert beziehungsweise validiert die folgende Funktion `build_interpreter` nun das von `build_lstm` gebaute neuronale Netz und liefert einen Interpreter-Funktion zurück. Diese erwartet als Eingabe eine Äußerung, transformiert diese anschließend mit dem angepassten  `IntentNumerizer`   und führt mit dem zuvor trainierten Netz die Intent Recognition durch.

In [25]:
BATCH_SIZE = 128
EPOCHS = 10

def build_interpreter(rules=RULES, units=UNITS, batch_size=128, epochs=EPOCHS):
    def interpreter(utterance):
        x = intent_numerizer.transform_utterance(tokenize(utterance))
        y, Y = model.predict(np.stack([x]))
        tag_idxs = np.argmax(Y[0], axis=1)
        intent_idx = np.argmax(y[0])
        return intent_numerizer.revert(intent_idx, tag_idxs)

    intent_numerizer, X_train, y_train, Y_train, X_test, y_test, Y_test = train_test_data(rules)
    model = build_bilstm(X_train.shape[2], y_train.shape[1], Y_train.shape[2], units)
    model.fit(x=X_train, y=[y_train, Y_train],
              validation_data=(X_test,[y_test, Y_test]),
              batch_size=batch_size, epochs=epochs)
    return interpreter


Und nun sind wir bereit zum Testen!

In [26]:
interpreter = build_interpreter()

Train on 702 samples, validate on 301 samples
Epoch 1/10
702/702 [==============================] - 3s 5ms/step - loss: 1.7896 - dense_2_loss: 1.0908 - time_distributed_1_loss: 0.6988 - val_loss: 0.5782 - val_dense_2_loss: 0.5287 - val_time_distributed_1_loss: 0.0495
Epoch 2/10
702/702 [==============================] - 2s 2ms/step - loss: 0.1985 - dense_2_loss: 0.1724 - time_distributed_1_loss: 0.0261 - val_loss: 0.0317 - val_dense_2_loss: 0.0154 - val_time_distributed_1_loss: 0.0164
Epoch 3/10
702/702 [==============================] - 2s 3ms/step - loss: 0.0221 - dense_2_loss: 0.0135 - time_distributed_1_loss: 0.0086 - val_loss: 0.0130 - val_dense_2_loss: 0.0083 - val_time_distributed_1_loss: 0.0047
Epoch 4/10
702/702 [==============================] - 2s 2ms/step - loss: 0.0047 - dense_2_loss: 0.0018 - time_distributed_1_loss: 0.0029 - val_loss: 0.0060 - val_dense_2_loss: 0.0039 - val_time_distributed_1_loss: 0.0021
Epoch 5/10
702/702 [==============================] - 2s 2ms/step 

In [27]:
interpreter('Welche ungefähre Temperatur war 1992 und 2018 in Sachsen')

('Frag_Temperatur',
 ['-',
  '-',
  '-',
  '-',
  'Zeit',
  '-',
  'Zeit',
  '-',
  'Ort',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-'])

Und jetzt kannt Du loslegen &mdash; der WetterBot kann noch nicht viel, ist aber nun recht einfach zu trainieren! Und mit der selbstgebauten Intent Recognition wird er bestimmt noch besser! Ein paar Ideen dazu gibt Dir das Notebook mit Aufgaben zu Intent Recognition.

_Viel Spaß und bis bald zu einer neuen Lektion vom codecentric.AI bootcamp!_